# Bibliotecas Usadas

In [ ]:
import csv
import sqlite3
import numpy as np
import pandas as pd
import datapane as dp
import plotly.offline as py
import plotly.express  as  px
import plotly.graph_objs as go
import chart_studio.tools as tls
import plotly.figure_factory as ff

from datetime import date
from plotly.subplots import make_subplots

# Tratamento dos dados

In [ ]:
df = pd.read_csv(r'C:\Users\lalad\Documents\Escola\UFMG\6ºperiodo\Visulização de dados\TP\Database\df_rides.csv', sep = ',', header = 'infer',encoding='utf-8',low_memory=False)

In [ ]:
def ordena(row):
    if row['ride_date'] == 'Monday':
        return 1
    if row['ride_date'] == 'Tuesday':
        return 2
    if row['ride_date'] == 'Wednesday':
        return 3
    if row['ride_date'] == 'Thursday':
        return 4
    if row['ride_date'] == 'Friday':
        return 5
    if row['ride_date'] == 'Saturday':
        return 6
    if row['ride_date'] == 'Sunday':
        return 7

In [ ]:
df['ride_date'] = pd.to_datetime(df['ride_date']).dt.strftime('%A')
#df['ride_date']

In [ ]:
df['ordem'] = df.apply (lambda row: ordena(row), axis=1)

In [ ]:
now = pd.Timestamp('now')
df['user_birthdate'] = pd.to_datetime(df['user_birthdate'], format='%Y-%m-%d')
df['user_birthdate'] = df['user_birthdate'].where(df['user_birthdate'] < now, df['user_birthdate'] -  np.timedelta64(100, 'Y'))
df['idade'] = (now - df['user_birthdate']).astype('<m8[Y]') 
#df['idade']

In [ ]:
#current_date = date.today().strftime('%Y-%m-%d')
#current_date
df['user_birthdate'] = pd.to_datetime(df['user_birthdate'], format='%Y-%m-%d')
#df['user_birthdate']


In [ ]:
df['id_station_start'] = df['station_start'].str.split('-').str[0].astype(int)
#df ["id_station_start"]

In [ ]:
df['name_station_start'] = df['station_start'].str.split('-').str[1]
#df ['name_station_start']

In [ ]:
df['id_station_end'] = df['station_end'].str.split('-').str[0].astype(int)
#df ['id_station_end']

In [ ]:
df['name_station_end'] = df['station_end'].str.split('-').str[1]
#df ['name_station_end']

In [ ]:
def classifica(row):
    if row['idade'] >= 0 and row['idade']<10:
        return 1
    if row['idade'] >=10 and row['idade']<20:
        return 2
    if row['idade'] >=20 and row['idade']<30:
        return 3
    if row['idade'] >=30 and row['idade']<40:
        return 4
    if row['idade'] >=40 and row['idade']<50:
        return 5
    if row['idade'] >=50 and row['idade']<60:
        return 6
    if row['idade'] >=60 and row['idade']<70:
        return 7
    if row['idade'] >=70 and row['idade']<80:
        return 8
    if row['idade'] >=80 and row['idade']<90:
        return 9
    if row['idade'] >=90 :
        return 10

In [ ]:
df['classifica'] = df.apply (lambda row: classifica(row), axis=1)

In [ ]:
def trans(row):
    if row['ride_late'] == 0.0:
        return '> 1hr'
    if row['ride_late'] == 1.0:
        return '< ou = 1hr'

In [ ]:
df['new'] = df.apply (lambda row: trans(row), axis=1)

In [ ]:
df

# Cria o banco de dados

In [ ]:
#Cria o banco bike
conn = sqlite3.connect('bike.db')
cursor = conn.cursor()

#Coloca a tabela tab no banco
Table1 = df
Table1.to_sql('tab',conn,if_exists = 'fail',index = False)
conn.close()

# Visualizações

<h3> Questões demográficas 

<p> 1) Qual a proporção de homens e mulheres que usam o serviço?

In [ ]:
conn = sqlite3.connect('bike.db')
cursor = conn.cursor()

sql = """select user_gender, count(user_gender)
from tab
where user_gender <> "None"
group by user_gender
"""

cursor = conn.execute(sql)
rows = cursor.fetchall()
conn.close()
#rows

In [ ]:
a = rows[0][1] / (rows[0][1] + rows[1][1]) * 100
b = rows[1][1] / (rows[0][1] + rows[1][1]) * 100

In [ ]:
data = [['M',b], ['F', a]] 
dataframe = pd.DataFrame(data, columns = ['Sexo', 'Porcentagem']) 
#dataframe

In [ ]:
fig  =  px . bar (dataframe,  x = dataframe['Sexo'] ,  y = dataframe['Porcentagem'], title = 'Parte e todo: sexo', color = dataframe['Sexo'])
fig.update_layout(yaxis=dict(range=[0,100]))
fig . show ()

<p>2) Qual o perfil etário dos usuários

In [ ]:
conn = sqlite3.connect('bike.db')
cursor = conn.cursor()

sql = """select count(user_gender)
from tab
where user_gender = 'M'
group by classifica
"""

cursor = conn.execute(sql)
rows = cursor.fetchall()
conn.close()
#rows

In [ ]:
men_bins = np.array(rows)
men_bins = men_bins.reshape(10,)

In [ ]:
conn = sqlite3.connect('bike.db')
cursor = conn.cursor()


sql = """select count(user_gender)
from tab
where user_gender = 'F' and idade <> "None"
group by classifica
"""
cursor = conn.execute(sql)
rows = cursor.fetchall()
conn.close()
#rows

In [ ]:
women_bins = np.array(rows)
women_bins = women_bins *(-1)
women_bins = women_bins.reshape(10,)

In [ ]:

y = list(range(0, 100, 10))

layout = go.Layout(yaxis=go.layout.YAxis(title='Idade'),
                   xaxis=go.layout.XAxis(
                       range=[-120000, 120000],
                       tickvals=[-100000, -50000,-10000, 0,10000, 50000, 100000],
                       ticktext=[100000, 50000,10000, 0,10000, 50000, 100000],
                       title='Quantidade de pessoas'),
                   barmode='overlay',
                   bargap=0.1,
                   title=go.layout.Title(text='Pirâmide etária dos usuários'))

data = [go.Bar(y=y,
               x=men_bins,
               orientation='h',
               name='Homem',
               hoverinfo='x',
               marker=dict(color='blue')
               ),
        go.Bar(y=y,
               x=women_bins,
               orientation='h',
               name='Mulher',
               text=-1 * women_bins.astype('int'),
               hoverinfo='text',
               marker=dict(color='red')
               )]

py.iplot(dict(data=data, layout=layout), filename='EXAMPLES/bar_pyramid')

<p>3) Qual a distribuição de idade por sexo? 

In [ ]:
conn = sqlite3.connect('bike.db')
cursor = conn.cursor()

sql = """select user_gender, idade
from tab
where user_gender <> "None"
"""

cursor = conn.execute(sql)
rows = cursor.fetchall()
conn.close()
#rows

In [ ]:
#Criando um dataframe com pandas

sexo = []
idade = []

for element in rows:
  sexo.append(element[0])
  idade.append(element[1])

dic = {
    'Sexo': sexo,
    'Idade': idade,
    
}
#dic
#Visualização do Data Frame
dataframe = pd.DataFrame(dic)
#dataframe

In [ ]:
fig = px.box( x = dataframe['Sexo'], y = dataframe['Idade'], color= dataframe['Sexo'],title = 'Distribuição de sexo por idade')
fig.show()

In [ ]:
fig.update_layout(xaxis_title='Sexo',yaxis_title="Idade")

<h3>Questões perfil de consumo

<p>1) Quais os padrões observados entre a idade e o dia da semana?

In [ ]:
conn = sqlite3.connect('bike.db')
cursor = conn.cursor()

sql = """select idade, ride_date
from tab
where idade between 0 and 17
order by ordem 
"""

cursor = conn.execute(sql)
rows = cursor.fetchall()
conn.close()
#rows

In [ ]:
#Criando um dataframe com pandas

idade = []
dia = []

for element in rows:
  idade.append(element[0])
  dia.append(element[1])

dic = {
    'Idade': idade,
    'Week day': dia,
    
}
#dic
#Visualização do Data Frame
dataframe1 = pd.DataFrame(dic)
#dataframe

In [ ]:
fig = px.box( x = dataframe1['Idade'], y = dataframe1['Week day'], color_discrete_sequence=['#6100A3'],title = 'Distribuição idade por dia da semana')
#fig.show()

In [ ]:
#A consulta SQL deve ser ordenada de forma decrescente. Alrerar a consulta acima e as próximas
#fig.update_layout(xaxis_title='Idade',yaxis_title="Dia da semana")

In [ ]:
conn = sqlite3.connect('bike.db')
cursor = conn.cursor()

sql = """select idade, ride_date
from tab
where idade between 18 and 59 
order by ordem 
"""

cursor = conn.execute(sql)
rows = cursor.fetchall()
conn.close()
#rows

In [ ]:
#Criando um dataframe com pandas

idade = []
dia = []

for element in rows:
  idade.append(element[0])
  dia.append(element[1])

dic = {
    'Idade': idade,
    'Week day': dia,
    
}
#dic
#Visualização do Data Frame
dataframe2 = pd.DataFrame(dic)
#dataframe

In [ ]:
fig = px.box( x = dataframe2['Idade'], y = dataframe2['Week day'], color_discrete_sequence=['#6100A3'])
#fig.show()

In [ ]:
#fig.update_layout(xaxis_title='Idade',yaxis_title="Dia da semana")

In [ ]:
conn = sqlite3.connect('bike.db')
cursor = conn.cursor()

sql = """select idade, ride_date
from tab
where idade >= 60
order by ordem
"""

cursor = conn.execute(sql)
rows = cursor.fetchall()
conn.close()
#rows

In [ ]:
#Criando um dataframe com pandas

idade = []
dia = []

for element in rows:
  idade.append(element[0])
  dia.append(element[1])

dic = {
    'Idade': idade,
    'Week day': dia,
    
}
#dic
#Visualização do Data Frame
dataframe3 = pd.DataFrame(dic)
#dataframe

In [ ]:
fig = px.box( x = dataframe3['Idade'], y = dataframe3['Week day'],color_discrete_sequence=['#6100A3'])
#fig.show()

In [ ]:
#fig.update_layout(xaxis_title='Idade',yaxis_title="Dia da semana")

In [ ]:
fig = make_subplots(rows=1, cols=3)

fig.add_trace(
    go.Box( x = dataframe1['Week day'], y = dataframe1['Idade'], marker_color='#B00983', name = 'Pessoas entre 0 e 17 anos'),
    row=1, col=1
)

fig.add_trace(
    go.Box( x = dataframe2['Week day'], y = dataframe2['Idade'], marker_color='#4309BA' ,name = 'Pessoas entre 18 e 59 anos'),
    row=1, col=2
)

fig.add_trace(
    go.Box(x = dataframe3['Week day'], y = dataframe3['Idade'],marker_color='#6100A3', name = 'Pessoas com 60 ou mais'),
    row=1, col=3
)

fig.update_layout(height=500, width=1000, title_text='Distribuição idade por dia da semana')
fig.show()

<p> 2) Quais os padrões observados entre o sexo da pessoa e o dia da semana?

<p> Vide Power BI

In [ ]:
conn = sqlite3.connect('bike.db')
cursor = conn.cursor()

sql = """select ordem, ride_date, user_gender, count(user_gender)
from tab
where user_gender = 'F'
group by ride_date
UNION
select ordem, ride_date, user_gender, count(user_gender)
from tab
where user_gender = 'M'
group by ride_date
order by ordem
"""

cursor = conn.execute(sql)
rows = cursor.fetchall()
conn.close()
#rows

In [ ]:
vetor1 = []
vetor2 = []
const = 3
i = 0
j = i+1
flag = 0 
while(j<14):
    if(flag == 0):
        vetor1.append(rows[i][const] / (rows[i][const] + rows[j][const]) * 100)
        flag = 1
    else:
        vetor2.append(100-vetor[i-1])
        flag = 0
    i += 1
    j = i+1
vetor.append(100-vetor[i-1])
#vetor

In [ ]:
matriz1 = [] # lista vazia
matriz2 = [] 
flag = 0
for i in range(14):
   # cria a linha i
    linha = [] # lista vazia
    if(flag % 2 == 0):
        linha.append(rows[i][1])
        linha.append("F")
        linha.append(vetor[i])
        matriz1.append(linha)
    else:
        linha.append(rows[i][1])
        linha.append("M")
        linha.append(vetor[i])
        matriz2.append(linha)
    flag += 1
    # coloque linha na matriz


#matriz
dataframe1 = pd.DataFrame(matriz1, columns = ['Dia','Sexo', 'Porcentagem']) 
dataframe2 = pd.DataFrame(matriz2, columns = ['Dia','Sexo', 'Porcentagem']) 
#dataframe

In [ ]:
fig = go.Figure(go.Bar(x=dataframe2['Dia'], y=dataframe1['Porcentagem'], name='Mulher',marker_color='#EF553B'))
fig.add_trace(go.Bar(x=dataframe2['Dia'], y=dataframe2['Porcentagem'], name='Homem',marker_color='#636EFA'))

fig.update_layout(barmode='stack',yaxis=dict(range=[0,100]),title_text='Parte e todo: Dia da semana por sexo')
fig.show()

<p> 3) Quais os dias da semana as pessoas mais usam bicicletas?

In [ ]:
conn = sqlite3.connect('bike.db')
cursor = conn.cursor()

sql = """select ride_date, count(ride_date)
from tab
group by ride_date
order by ordem
"""

cursor = conn.execute(sql)
rows = cursor.fetchall()
conn.close()
#rows
#Criando um dataframe com pandas

dia = []
ordem = []

for element in rows:
  dia.append(element[0])
  ordem.append(element[1])

dic = {
    'Week day': dia,
    'Count': ordem,
}
#dic
#Visualização do Data Frame
dataframe = pd.DataFrame(dic)
#dataframe

In [ ]:
#fig  =  px . bar (dataframe,  x = dataframe['Week day'] ,  y = dataframe['Count'], title = 'Bar plot: Week day vs. Count', color_discrete_sequence=['#6100A3'] ) 
#fig . show ()

In [ ]:
fig  =  px . bar (dataframe,  x = dataframe['Count'] ,  y = dataframe['Week day'], color_discrete_sequence=['#6100A3'],  orientation='h' ) 
#fig . show ()

In [ ]:

fig.update_layout(xaxis_title='Número de pessoas',yaxis_title='Dia da semana',yaxis={'categoryorder':'total ascending'})
fig.show()

<p> 4) Como é o tempo de duração das corridas em relação ao dia da semana?

In [ ]:
conn = sqlite3.connect('bike.db')
cursor = conn.cursor()

sql = """select ordem,ride_date, count(ride_late),new
from tab
where ride_late = '0'
group by ride_date
UNION
select ordem,ride_date, count(ride_late), new
from tab
where ride_late = '1'
group by ride_date
order by ordem
"""

cursor = conn.execute(sql)
rows = cursor.fetchall()
conn.close()
#rows

In [ ]:
vetor = []
const = 2
i = 0
j = i+1
flag = 0 
while(j<14):
    if(flag == 0):
        vetor.append(rows[i][const] / (rows[i][const] + rows[j][const]) * 100)
        flag = 1
    else:
        vetor.append(100-vetor[i-1])
        flag = 0
    i += 1
    j = i+1
vetor.append(100-vetor[i-1])
#vetor

In [ ]:
matriz = [] # lista vazia
flag = 0
for i in range(14):
   # cria a linha i
    linha = [] # lista vazia
    if(flag % 2 == 0):
        linha.append(rows[i][1])
        linha.append("< ou = 1hr")
        linha.append(vetor[i])
    else:
        linha.append(rows[i][1])
        linha.append("> 1hr")
        linha.append(vetor[i])
    matriz.append(linha)
    flag += 1
    # coloque linha na matriz


#matriz
dataframe = pd.DataFrame(matriz, columns = ['Dia','Duração', 'Porcentagem']) 
#dataframe

In [ ]:
fig = px.bar(dataframe, x='Dia da semana', y='Porcentagem',color='Duração',title = 'Parte e todo: duração das corridas',color_discrete_sequence=['#CEF000', '#6100A3'])
fig.update_layout(yaxis=dict(range=[0,100]))
fig . show ()

In [ ]:
fig = px.violin(df, x='ride_date', y='ride_duration',color_discrete_sequence=['#6100A3'])
fig.show()

In [ ]:
conn = sqlite3.connect('bike.db')
cursor = conn.cursor()

sql = """select ordem, ride_date, ride_duration
from tab
where ride_duration < 100
order by ordem
"""

cursor = conn.execute(sql)
rows = cursor.fetchall()
conn.close()
#rows

In [ ]:
#Criando um dataframe com pandas

ordem = []
dia = []
duracao = []

for element in rows:
  ordem.append(element[0])
  dia.append(element[1])
  duracao.append(element[2])

dic = {
    'Ordem': ordem,
    'Dia': dia,
    'Duração':duracao,
    
}
#dic
#Visualização do Data Frame
dataframe = pd.DataFrame(dic)
#dataframe

In [ ]:
fig = px.violin(dataframe, x='Dia', y='Duração',color_discrete_sequence=['#6100A3'])
fig.show()

In [ ]:
conn = sqlite3.connect('bike.db')
cursor = conn.cursor()

sql = """select ordem, ride_date, ride_duration
from tab
where ride_duration > 100 AND ride_duration < 200
order by ordem
"""

cursor = conn.execute(sql)
rows = cursor.fetchall()
conn.close()
#rows

<h3> Questões geográficas

<p>1) Onde as estações estão localizadas?

<p> Vide Power BI

<p> 2) Quais as estações mais usadas? Para pegar a bicicleta emprestada e devolve-la respectivamente.

<p> Vide Power BI

In [ ]:
conn = sqlite3.connect('bike.db')
cursor = conn.cursor()

sql = """select id_station_start, name_station_start, count(name_station_start)
from tab
group by name_station_start
order by count(name_station_start)
"""

cursor = conn.execute(sql)
rows = cursor.fetchall()
conn.close()
#rows
#Criando um dataframe com pandas

identificador = []
nome = []
count = []

for element in rows:
  identificador.append(element[0])
  nome.append(element[1])
  count.append(element[2])

dic = {
    'id': identificador,
    'Nome':nome,
    'Count': count,
}
#dic
#Visualização do Data Frame
dataframe1 = pd.DataFrame(dic)
#dataframe1

In [ ]:
fig  =  px . bar (dataframe1,  x = dataframe1['Count'] ,  y = dataframe1['Nome'], title = 'Bar plot: Count vs. Nome', color_discrete_sequence=['#6100A3'],  orientation='h') 
fig.update_layout( yaxis={'categoryorder':'total ascending'})
fig.show()

In [ ]:
conn = sqlite3.connect('bike.db')
cursor = conn.cursor()

sql = """select id_station_start, name_station_start, count(name_station_start)
from tab
group by name_station_start
having count(name_station_start) > 11000
order by count(name_station_start)
"""

cursor = conn.execute(sql)
rows = cursor.fetchall()
conn.close()
#rows
#Criando um dataframe com pandas

identificador = []
nome = []
count = []

for element in rows:
  identificador.append(element[0])
  nome.append(element[1])
  count.append(element[2])

dic = {
    'id': identificador,
    'Nome':nome,
    'Count': count,
}
#dic
#Visualização do Data Frame
dataframe1 = pd.DataFrame(dic)
#dataframe

In [ ]:
fig  =  px . bar (dataframe1,  x = dataframe1['Count'] ,  y = dataframe1['Nome'], title = 'Bar plot: Count vs. Nome', color_discrete_sequence=['#6100A3'],  orientation='h') 
fig.update_layout( yaxis={'categoryorder':'total ascending'})
fig.show()

In [ ]:
conn = sqlite3.connect('bike.db')
cursor = conn.cursor()

sql = """select id_station_end, name_station_end, count(name_station_end)
from tab
group by name_station_end
order by count(name_station_end)
"""
cursor = conn.execute(sql)
rows = cursor.fetchall()
conn.close()
#rows
#Criando um dataframe com pandas

identificador = []
nome = []
count = []

for element in rows:
  identificador.append(element[0])
  nome.append(element[1])
  count.append(element[2])

dic = {
    'id': identificador,
    'Nome':nome,
    'Count': count,
}
#dic
#Visualização do Data Frame
dataframe2 = pd.DataFrame(dic)
#dataframe

In [ ]:
fig  =  px . bar (dataframe2,  x = dataframe2['Count'] ,  y = dataframe2['Nome'], title = 'Bar plot: Count vs. Nome', color_discrete_sequence=['#6100A3'],  orientation='h') 
fig.update_layout( yaxis={'categoryorder':'total ascending'})
fig.show()

In [ ]:
conn = sqlite3.connect('bike.db')
cursor = conn.cursor()

sql = """select id_station_end, name_station_end, count(name_station_end)
from tab
group by name_station_end
having count(name_station_end) > 11000
order by count(name_station_end)
"""
cursor = conn.execute(sql)
rows = cursor.fetchall()
conn.close()
#rows
#Criando um dataframe com pandas

identificador = []
nome = []
count = []

for element in rows:
  identificador.append(element[0])
  nome.append(element[1])
  count.append(element[2])

dic = {
    'id': identificador,
    'Nome':nome,
    'Count': count,
}
#dic
#Visualização do Data Frame
dataframe2 = pd.DataFrame(dic)
#dataframe

In [ ]:
fig  =  px . bar (dataframe2,  x = dataframe2['Count'] ,  y = dataframe2['Nome'], title = 'Bar plot: Count vs. Nome', color_discrete_sequence=['#6100A3'],  orientation='h') 
fig.update_layout( yaxis={'categoryorder':'total ascending'})
fig.show()

In [ ]:
fig = make_subplots(rows=2, cols=1)

fig.add_trace(
    go.Bar(x = dataframe1['Count'] ,  y = dataframe1['Nome'], marker_color='#B00983', name = 'Principais estações de início', orientation='h'),
        row=1, col=1
)

fig.add_trace(
    go.Bar(x = dataframe2['Count'] ,  y = dataframe2['Nome'], marker_color='#6100A3', name = 'Principais estações de fim', orientation='h'),
    row=2, col=1
)


fig.update_layout(height=500, width=1000,title_text='Ranking estações')
#fig.update_xaxes(tick0=0, dtick=5000)
fig.show()

In [ ]:
conn = sqlite3.connect('bike1.db')
cursor = conn.cursor()

sql = """select idade, count(name_station_start)
from tab
group by idade
order by idade
"""

cursor = conn.execute(sql)
rows = cursor.fetchall()
conn.close()
rows

In [ ]:
data_matrix = [['Data de nascimento'],
               ['2028-04-28'],
               ['2027-04-16'],
               ['2027-01-16'],
               ['2026-12-03'],
               ['2026-12-03']]

fig = ff.create_table(data_matrix)
fig.show()

In [ ]:
conn = sqlite3.connect('bike1.db')
cursor = conn.cursor()

sql = """select *
from tab
"""

db_df = pd.read_sql_query(sql, conn)
rows = cursor.fetchall()
conn.close()
db_df.to_csv('database.csv', index=False)

In [ ]:
conn = sqlite3.connect('bike1.db')
cursor = conn.cursor()

sql = """select ordem,ride_date, count(user_gender)
from tab
where user_gender = 'M'
group by ride_date
UNION
select ordem,ride_date, count(user_gender)
from tab
where user_gender = 'F'
group by ride_date
order by ordem
"""

cursor = conn.execute(sql)
rows = cursor.fetchall()
conn.close()
rows